# Notebook 4: Ajuste del Modelo SARIMAX de Retornos Logarítmicos, Pronósticos y Validación del Modelo
**Proyecto:** Análisis SARIMAX - Starbucks Corporation (SBUX)  
**Investigador:** Frankli Zeña Zeña (UNI)
___   

En este notebook construiremos el modelo SARIMAX óptimo para predecir el precio de cierre (`Close`) de Starbucks. 
Nuestro enfoque será iterativo:
1. **Búsqueda Automática:** Usaremos `Auto-ARIMA` para encontrar los hiperparámetros $(p, d, q) \times (P, D, Q)_m$.
2. **Evaluación de Exógenas:** Analizaremos el *Summary* del modelo para evaluar la significancia estadística ($P-valor$) de nuestras variables financieras y dummies.
3. **Poda (Feature Selection):** Eliminaremos las variables que no aporten poder predictivo ($P > 0.05$) para evitar el sobreajuste (overfitting).

## Carga y Split de Datos

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pmdarima as pm
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [3]:
ruta_archivo = '../data/transformed/sbux_master_sarimax.csv'
df = pd.read_csv(ruta_archivo, index_col='Fecha', parse_dates=True).dropna()
df

,Date,Adj Close,Volume,Vol_Avg_20,Vol_Anomaly,Log_Return,Margen_Operativo_%,Revenue,choque_estructural,shock_extremo,earnings,riesgo_pais,shock_costos
Fecha,,,,,,,,,,,,,
2021-03-31,2021-03-31,97.421112,6478400,7244615.0,False,-0.009110,14.811039,6668.0,0,0,0,0,0
2021-04-01,2021-04-01,97.519173,5793000,7173675.0,False,0.001006,14.811039,6668.0,0,0,0,0,0
2021-04-05,2021-04-05,98.981331,6913100,7241335.0,False,0.014882,14.811039,6668.0,0,0,0,0,0
2021-04-06,2021-04-06,100.880363,6745200,7322620.0,False,0.019004,14.811039,6668.0,0,0,0,0,0
2021-04-07,2021-04-07,100.916023,5629600,7328555.0,False,0.000353,14.811039,6668.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2026-02-09,2026-02-09,98.345779,7150600,11434955.0,False,-0.004737,8.984274,9915.1,1,0,0,0,0
2026-02-10,2026-02-10,96.905067,8543500,11497655.0,False,-0.014758,8.984274,9915.1,1,0,0,0,0
2026-02-11,2026-02-11,98.484879,6949100,11567015.0,False,0.016171,8.984274,9915.1,1,0,0,0,0


In [3]:
split_idx = int(len(df) * 0.8)

In [4]:
df_train = df[:split_idx]
df_test = df[split_idx:]

In [6]:
target_col = 'Log_Return'

y = df[target_col]

exog_cols = [
    'Margen_Operativo_%', 'Revenue', 
    'choque_estructural', 'shock_extremo', 
    'earnings', 'riesgo_pais', 'shock_costos'
]

X = df[exog_cols]

In [7]:

y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]

print(f"Datos listos. Train: {len(y_train)} obs. | Test: {len(y_test)} obs.")

Datos listos. Train: 980 obs. | Test: 245 obs.


## Paso 1: Búsqueda del Modelo Óptimo (Auto-ARIMA)

Del Notebook 3 sabemos que la serie original requiere una primera diferencia ($d=1$) para ser estacionaria. Además, identificamos un ciclo estacional de $m=6$.
Inyectaremos nuestra matriz `X_train` al algoritmo para que busque la mejor combinación de rezagos autorregresivos y medias móviles que minimice el Criterio de Información de Akaike (AIC), para ello utilizaremos Auto-ARIMA.  
El algoritmo iterará sobre múltiples combinaciones de $p$, $q$, $P$ y $Q$ para minimizar el **Criterio de Información de Akaike (AIC)**. El modelo con el AIC más bajo será nuestro modelo óptimo.

In [8]:
# %pip install pmdarima
import pmdarima as pm

In [10]:
print("Iniciando búsqueda Auto-ARIMA con Variables Exógenas...")

# Ejecutamos el Auto-ARIMA sobre nuestro conjunto de Entrenamiento (y_train, X_train)
modelo_auto = pm.auto_arima(
    y=y_train, 
    X=X_train,               # ¡Aquí entran tus finanzas y dummies!
    start_p=0, start_q=0,    # Empezamos desde cero
    max_p=2, max_q=2,        # Límite máximo para p y q (para no sobreajustar)
    d=0,                     # Sin diferencia YA ESTACIONAL confirmada por Dickey-Fuller
    seasonal=True,           # Activamos la estacionalidad
    m=21,                     # Ciclo estacional de 21 periodos
    start_P=0, start_Q=0,
    max_P=2, max_Q=2,
    D=1,                     # Diferencia estacional (usualmente 1 si hay estacionalidad fuerte)
    trace=True,              # Imprime el proceso paso a paso
    error_action='ignore',  
    suppress_warnings=True, 
    stepwise=True            # Búsqueda inteligente
)

print("\n ¡Búsqueda completada!")
print()

Iniciando búsqueda Auto-ARIMA con Variables Exógenas...
Performing stepwise search to minimize aic
 ARIMA(0,0,0)(0,1,0)[21] intercept   : AIC=-4265.418, Time=2.14 sec
 ARIMA(1,0,0)(1,1,0)[21] intercept   : AIC=-4529.012, Time=19.70 sec
 ARIMA(0,0,1)(0,1,1)[21] intercept   : AIC=-4656.522, Time=7.79 sec
 ARIMA(0,0,0)(0,1,0)[21]             : AIC=-4264.881, Time=3.93 sec
 ARIMA(0,0,1)(0,1,0)[21] intercept   : AIC=-4263.451, Time=2.57 sec
 ARIMA(0,0,1)(1,1,1)[21] intercept   : AIC=-4636.033, Time=17.95 sec
 ARIMA(0,0,1)(0,1,2)[21] intercept   : AIC=-4658.982, Time=50.01 sec
 ARIMA(0,0,1)(1,1,2)[21] intercept   : AIC=-4664.030, Time=54.43 sec
 ARIMA(0,0,1)(2,1,2)[21] intercept   : AIC=-4656.953, Time=55.22 sec
 ARIMA(0,0,1)(2,1,1)[21] intercept   : AIC=-4618.272, Time=59.98 sec
 ARIMA(0,0,0)(1,1,2)[21] intercept   : AIC=-4666.546, Time=56.99 sec
 ARIMA(0,0,0)(0,1,2)[21] intercept   : AIC=-4661.204, Time=40.16 sec
 ARIMA(0,0,0)(1,1,1)[21] intercept   : AIC=-4638.487, Time=17.76 sec
 ARIMA(0

## Paso 2: Ajuste SARIMAX y Resumen Estadístico (Summary)

Entrenamos el modelo formalmente usando `statsmodels` con los hiperparámetros ganadores. 
El objetivo principal de esta celda es obtener la tabla de coeficientes y enfocarnos en la columna `P>|z|` para nuestras variables exógenas.
* **Hipótesis Nula ($H_0$):** La variable exógena no tiene efecto sobre el precio de Starbucks.
* Si $P \le 0.05$: Rechazamos $H_0$. La variable se queda (es estadísticamente significativa).
* Si $P > 0.05$: Aceptamos $H_0$. La variable es ruido y deberá ser eliminada en la siguiente iteración.

In [17]:
orden_optimo = (0, 0, 1) 
orden_estacional = (0, 1, 2, 21) 

# Definimos el modelo maestro
modelo_sarimax = sm.tsa.statespace.SARIMAX(
    endog=y_train, 
    exog=X_train, 
    order=orden_optimo, 
    seasonal_order=orden_estacional,
    enforce_stationarity=False,
    enforce_invertibility=False
)

# Ajustamos el modelo
resultados = modelo_sarimax.fit(disp=False)

# Imprimimos la radiografía del modelo
print(resultados.summary())

                                        SARIMAX Results                                        
Dep. Variable:                              Log_Return   No. Observations:                  980
Model:             SARIMAX(0, 0, 1)x(0, 1, [1, 2], 21)   Log Likelihood                2290.944
Date:                                lu., 16 feb. 2026   AIC                          -4559.887
Time:                                         10:07:42   BIC                          -4506.879
Sample:                                              0   HQIC                         -4539.654
                                                 - 980                                         
Covariance Type:                                   opg                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Margen_Operativo_%     0.0014      0.000      3.251      0

> Se observa que tanto los factores ``Revenue``, ``riesgo_pais``, ``shock_costos``, ``MA(1)`` y ``MA estacional de orden 2`` no tienen mayor influencia en la estimación del **Precio**, es preciso retirarlas


In [18]:
exog_cols_2 = [ 
    'Margen_Operativo_%','choque_estructural',
    'shock_extremo','earnings'
]

X_2 = df[exog_cols_2]

In [19]:
X_train_2, X_test_2 = X_2.iloc[:split_idx], X_2.iloc[split_idx:]

print(f"Datos listos. Train: {len(y_train)} obs. | Test: {len(y_test)} obs.")

Datos listos. Train: 980 obs. | Test: 245 obs.


In [21]:
orden_optimo = (0, 0, 0) 
orden_estacional = (0, 1, 1, 21) 

# Definimos el modelo maestro
modelo_sarimax = sm.tsa.statespace.SARIMAX(
    endog=y_train, 
    exog=X_train_2, 
    order=orden_optimo, 
    seasonal_order=orden_estacional,
    enforce_stationarity=False,
    enforce_invertibility=False
)

# Ajustamos el modelo
resultados = modelo_sarimax.fit(disp=False)

# Imprimimos la radiografía del modelo
print(resultados.summary())

                                 SARIMAX Results                                  
Dep. Variable:                 Log_Return   No. Observations:                  980
Model:             SARIMAX(0, 1, [1], 21)   Log Likelihood                2377.614
Date:                   lu., 16 feb. 2026   AIC                          -4743.227
Time:                            10:11:30   BIC                          -4714.171
Sample:                                 0   HQIC                         -4732.149
                                    - 980                                         
Covariance Type:                      opg                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
Margen_Operativo_%     0.0002      0.000      0.618      0.537      -0.000       0.001
choque_estructural     0.0022      0.002      1.237      0.216      -0.001 

In [ ]:
exog_cols_3 = [ 
    'choque_estructural','shock_extremo','earnings'
]

X_3 = df[exog_cols_3]

In [23]:
X_train_3, X_test_3 = X_3.iloc[:split_idx], X_3.iloc[split_idx:]

In [24]:
orden_optimo = (0, 0, 0) 
orden_estacional = (0, 1, 1, 21) 

# Definimos el modelo maestro
modelo_sarimax = sm.tsa.statespace.SARIMAX(
    endog=y_train, 
    exog=X_train_3, 
    order=orden_optimo, 
    seasonal_order=orden_estacional,
    enforce_stationarity=False,
    enforce_invertibility=False
)

# Ajustamos el modelo
resultados = modelo_sarimax.fit(disp=False)

# Imprimimos la radiografía del modelo
print(resultados.summary())

                                 SARIMAX Results                                  
Dep. Variable:                 Log_Return   No. Observations:                  980
Model:             SARIMAX(0, 1, [1], 21)   Log Likelihood                2377.347
Date:                   lu., 16 feb. 2026   AIC                          -4744.694
Time:                            10:12:59   BIC                          -4720.481
Sample:                                 0   HQIC                         -4735.463
                                    - 980                                         
Covariance Type:                      opg                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------
choque_estructural     0.0017      0.001      1.254      0.210      -0.001       0.004
shock_extremo         -0.0959      0.003    -28.753      0.000      -0.102 

In [26]:
exog_cols_4 = [ 
    'shock_extremo','earnings'
]

X_4 = df[exog_cols_4]

In [27]:
X_train_4, X_test_4 = X_4.iloc[:split_idx], X_4.iloc[split_idx:]

In [28]:
orden_optimo = (0, 0, 0) 
orden_estacional = (0, 1, 1, 21) 

# Definimos el modelo maestro
modelo_sarimax = sm.tsa.statespace.SARIMAX(
    endog=y_train, 
    exog=X_train_4, 
    order=orden_optimo, 
    seasonal_order=orden_estacional,
    enforce_stationarity=False,
    enforce_invertibility=False
)

# Ajustamos el modelo
resultados = modelo_sarimax.fit(disp=False)

# Imprimimos la radiografía del modelo
print(resultados.summary())

                                 SARIMAX Results                                  
Dep. Variable:                 Log_Return   No. Observations:                  980
Model:             SARIMAX(0, 1, [1], 21)   Log Likelihood                2376.698
Date:                   lu., 16 feb. 2026   AIC                          -4745.396
Time:                            10:14:53   BIC                          -4726.025
Sample:                                 0   HQIC                         -4738.010
                                    - 980                                         
Covariance Type:                      opg                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
shock_extremo    -0.0956      0.003    -28.666      0.000      -0.102      -0.089
earnings          0.0088      0.002      4.215      0.000       0.005       0.013
ma.S.L21

## Paso 3: Predicción (Forecasting) sobre el Conjunto Test

Una vez ajustado y depurado nuestro modelo SARIMAX, procederemos a generar las predicciones para el horizonte de tiempo que corresponde a nuestro conjunto `Test` (el 20% más reciente de los datos). 
Para ello, el algoritmo requiere que le suministremos la matriz de variables exógenas de ese periodo (`X_test`). Además de la predicción puntual, calcularemos los Intervalos de Confianza al 95%.

In [41]:
import plotly.graph_objects as go

In [29]:
# Generar la predicción indicando cuántos pasos a futuro (len de y_test) y entregando las variables exógenas de ese periodo temporal
forecast = resultados.get_forecast(steps=len(df_test), exog=X_test_4)
forecast_index = df_test.index
forecast_values = forecast.predicted_mean

# Intervalos de Confianza al 95%
conf_int = forecast.conf_int()

print(f"Total de días predichos: {len(forecast_values)}")

Total de días predichos: 245


### Paso 3.1: Visualización: Realidad vs. Predicción

Graficaremos el comportamiento real de la acción (`y_test`) frente a la proyección de nuestro modelo (`pred_media`), incluyendo la banda de los intervalos de confianza. Para dar contexto visual sin comprimir la gráfica, incluiremos solo los últimos 250 días del conjunto de entrenamiento.

In [32]:
import plotly.graph_objects as go

In [33]:
# Crear figura interactiva
fig = go.Figure()

# Datos reales
fig.add_trace(go.Scatter(
    x=y.index,
    y=y,
    mode='lines',
    name='Datos reales',
    line=dict(color='blue')
))

# Predicciones
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=forecast_values,
    mode='lines',
    name='Predicciones',
    line=dict(color='red')
))

# Intervalo de confianza (banda superior)
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=conf_int.iloc[:, 1],
    mode='lines',
    line=dict(width=0),
    showlegend=False
))

# Intervalo de confianza (banda inferior con relleno)
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=conf_int.iloc[:, 0],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(255, 0, 0, 0.3)',
    line=dict(width=0),
    name='Intervalo de Confianza'
))

# Personalización del diseño
fig.update_layout(
    title='Predicciones vs Datos Reales',
    xaxis_title='Fecha',
    yaxis_title='Valor',
    hovermode='x unified',
    template='plotly_white',
    width=1200,
    height=500
)

fig.show()

In [55]:
# Último precio real antes del período de test
last_train_price = df_train['Adj Close'].iloc[-1]

# Reconstrucción iterativa de precios predichos
predicted_prices = [last_train_price]

for r in forecast_values:
    next_price = predicted_prices[-1] * np.exp(r)
    print(f"Predicción log-return: {r:.4f} | Precio reconstruido: {next_price:.2f}")
    predicted_prices.append(next_price)

# Eliminamos el primer valor (era solo el punto base)
predicted_prices = predicted_prices[1:]

predicted_prices = pd.Series(predicted_prices, index=forecast_index)

conf_int_price_lower = last_train_price * np.exp(conf_int.iloc[:, 0].cumsum())
conf_int_price_upper = last_train_price * np.exp(conf_int.iloc[:, 1].cumsum())

predicted_prices

Predicción log-return: -0.0048 | Precio reconstruido: 109.62
Predicción log-return: -0.0009 | Precio reconstruido: 109.53
Predicción log-return: 0.0027 | Precio reconstruido: 109.82
Predicción log-return: 0.0057 | Precio reconstruido: 110.45
Predicción log-return: -0.0005 | Precio reconstruido: 110.40
Predicción log-return: 0.0001 | Precio reconstruido: 110.41
Predicción log-return: -0.0040 | Precio reconstruido: 109.97
Predicción log-return: 0.0004 | Precio reconstruido: 110.01
Predicción log-return: -0.0042 | Precio reconstruido: 109.55
Predicción log-return: 0.0010 | Precio reconstruido: 109.66
Predicción log-return: -0.0011 | Precio reconstruido: 109.53
Predicción log-return: -0.0006 | Precio reconstruido: 109.47
Predicción log-return: 0.0014 | Precio reconstruido: 109.63
Predicción log-return: 0.0047 | Precio reconstruido: 110.15
Predicción log-return: 0.0073 | Precio reconstruido: 110.95
Predicción log-return: 0.0014 | Precio reconstruido: 111.11
Predicción log-return: -0.0001 | 

Fecha
2025-02-25    109.620420
2025-02-26    109.526652
2025-02-27    109.824107
2025-02-28    110.454389
2025-03-03    110.398093
                 ...    
2026-02-09    136.878659
2026-02-10    136.728635
2026-02-11    136.648839
2026-02-12    136.843541
2026-02-13    137.492310
Length: 245, dtype: float64

In [57]:
import plotly.graph_objects as go

fig = go.Figure()

# Precio real entrenamiento
fig.add_trace(go.Scatter(
    x=df.index,
    y=df['Adj Close'],
    mode='lines',
    name='Train Adj Close',
    line=dict(color='black')
))

# Precio predicho reconstruido
fig.add_trace(go.Scatter(
    x=forecast_index,
    y=predicted_prices,
    mode='lines',
    name='Predicción Adj Close',
    line=dict(color='red')
))

fig.update_layout(
    title='Predicción SARIMAX - Precio Adj Close reconstruido',
    xaxis_title='Fecha',
    yaxis_title='Precio',
    hovermode='x unified',
    template='plotly_white',
    width=1200,
    height=500
)

fig.show()


## Paso 4: Cálculo de Métricas de Error y Sesgo

Para cuantificar científicamente el rendimiento de nuestro modelo en el Camino 1 (Precio Absoluto), calcularemos los siguientes indicadores sobre el conjunto de prueba:
* **MSE (Mean Squared Error):** Penaliza fuertemente los errores grandes.
* **RMSE (Root Mean Squared Error):** El error promedio expresado en las mismas unidades que la variable (Dólares).
* **MAE (Mean Absolute Error):** La magnitud promedio de los errores sin considerar la dirección.
* **MAPE (Mean Absolute Percentage Error):** El error promedio en formato porcentual (ideal para comparar con otros modelos o activos).
* **Sesgo (Bias):** Promedio de los errores. Indica si el modelo tiende a sobreestimar (sesgo negativo) o subestimar (sesgo positivo) sistemáticamente.

In [51]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

# Calculamos los errores absolutos (Realidad - Predicción)
errores = y_test - forecast_values

# 1. Mean Squared Error (MSE)
mse = mean_squared_error(y_test, forecast_values)
# 2. Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)
# 3. Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, forecast_values)
# 4. Mean Absolute Percentage Error (MAPE)
mape = mean_absolute_percentage_error(y_test, forecast_values) * 100
# 5. Sesgo (Bias) - Promedio aritmético de los errores
sesgo = errores.mean()

# Tabla de resultados
print("-" * 40)
print("================== INDICADORES DE RENDIMIENTO ==================")
print("-" * 40)
print(f"🔹 MSE   : {mse:.4f}")
print(f"🔹 RMSE  : ${rmse:.2f} USD (Error promedio por acción)")
print(f"🔹 MAE   : ${mae:.2f} USD (Desviación absoluta promedio)")
print(f"🔹 MAPE  : {mape:.2f}% (Error porcentual)")
print(f"🔹 Sesgo : {sesgo:.4f}")
print("-" * 40)

----------------------------------------
================== INDICADORES DE RENDIMIENTO ==================
----------------------------------------
🔹 MSE   : 0.0005
🔹 RMSE  : $0.02 USD (Error promedio por acción)
🔹 MAE   : $0.02 USD (Desviación absoluta promedio)
🔹 MAPE  : 125.48% (Error porcentual)
🔹 Sesgo : nan
----------------------------------------
